In [14]:
import pandas as pd
import numpy as np
import os

# Define the file paths
input_path = r"C:\Users\dhvani\Downloads\WEB Sitesi yeniden.xlsx"
output_path = r"C:\Users\dhvani\Downloads\Final-str_Data.xlsx"

print("Setup complete. Paths defined.")

Setup complete. Paths defined.


In [15]:
if not os.path.exists(input_path):
    print("Error: File not found. Check the path in Cell 1.")
else:
    # Load skipping junk rows (Header is at Row 10, index 9)
    df = pd.read_excel(input_path, header=9)
    
    # Select columns by Index:
    # 2: Industry (Oil & Gas, etc.)
    # 15: Item Name
    # 25, 26: Links
    # 28, 29: Contacts
    df_clean = df.iloc[:, [2, 15, 25, 26, 28, 29]].copy()
    df_clean.columns = ['Industry', 'Item_Name', 'Primary_Link', 'Secondary_Link', 'Whatsapp', 'Email']
    
    print("File loaded. Raw shape:", df_clean.shape)
    print("Sample of raw Industry column:", df_clean['Industry'].unique())

File loaded. Raw shape: (87, 6)
Sample of raw Industry column: [nan 'OIL & GAS Pipings systems' 'DAIRY & FOOD' 'INDUSTRIAL '
 'CONSULTING &  SERVICES']


In [16]:
# 1. Clean invisible spaces
df_clean['Item_Name'] = df_clean['Item_Name'].astype(str).str.strip()
df_clean['Industry'] = df_clean['Industry'].astype(str).str.strip().replace('nan', np.nan)

# 2. Fill Industry DOWN (so "Oil & Gas" applies to all items below it)
df_clean['Industry'] = df_clean['Industry'].ffill()

# 3. Fill Contacts DOWN & UP
df_clean[['Whatsapp', 'Email']] = df_clean[['Whatsapp', 'Email']].ffill().bfill()

# 4. Remove rows that have no Item Name
df_clean = df_clean[df_clean['Item_Name'] != 'nan']

print("Data filled and sanitized.")
print(df_clean[['Industry', 'Item_Name']].head(10))

Data filled and sanitized.
  Industry                                     Item_Name
0      NaN  NG FACTORY PIPELINES AND SKIDS INSTALLATIONS
1      NaN    LNG STORAGE TANKS AND SYSTEM INSTALLATIONS
2      NaN                 NITROGEN  & OXYGEN GENERATORS
3      NaN                                      PRODUCTS
4      NaN                                         Pipes
5      NaN                             Valves & Fittings
6      NaN                          Flexible connections
7      NaN                                       Filters
8      NaN                           Pressure Regulators
9      NaN                                    Gas Meters


In [17]:
types = []
rows_to_drop = []
current_type = "Project" # Default

for i, row in df_clean.iterrows():
    name = row['Item_Name'].upper()
    
    # Check if the row is actually a Header
    if "PRODUCT" in name:
        current_type = "Product"
        rows_to_drop.append(i) # Mark header for deletion
    elif "PROJECT" in name:
        current_type = "Project"
        rows_to_drop.append(i)
    elif "SERVICE" in name:
        current_type = "Service"
        rows_to_drop.append(i)
        
    types.append(current_type)

df_clean['Type'] = types

# Remove the header rows now that we used them
df_clean.drop(rows_to_drop, inplace=True)

print("Types assigned (Project/Product/Service).")

Types assigned (Project/Product/Service).


In [18]:
# Reorder columns
final_cols = ['Industry', 'Type', 'Item_Name', 'Primary_Link', 'Secondary_Link', 'Whatsapp', 'Email']
df_final = df_clean[final_cols]

# Save
try:
    df_final.to_excel(output_path, index=False)
    print(f"Success! File saved to: {output_path}")
    print("\n--- Final Preview ---")
    print(df_final.head())
except PermissionError:
    print("Error: Close the Excel file if it's open!")

Success! File saved to: C:\Users\dhvani\Downloads\Final-str_Data.xlsx

--- Final Preview ---
  Industry     Type                                     Item_Name  \
0      NaN  Project  NG FACTORY PIPELINES AND SKIDS INSTALLATIONS   
1      NaN  Project    LNG STORAGE TANKS AND SYSTEM INSTALLATIONS   
2      NaN  Project                 NITROGEN  & OXYGEN GENERATORS   
4      NaN  Product                                         Pipes   
5      NaN  Product                             Valves & Fittings   

                                        Primary_Link  \
0                         https://pakmon.vercel.app/   
1                         https://pakmon.vercel.app/   
2  https://www.forevergasindustry.com/en-us/products   
4                                                NaN   
5             https://www.g-bee.de/products#optionen   

                                      Secondary_Link               Whatsapp  \
0                                                NaN  ENQUIRY BOX_ Whats UP 

In [19]:
import pandas as pd
import numpy as np

# 1. Load the file you just checked
input_path = r"C:\Users\dhvani\Downloads\Final-str_Data.xlsx"
output_path = r"C:\Users\dhvani\Downloads\Corrected_Final_Data.xlsx"

# Load it
df = pd.read_excel(input_path)

# 2. Fix Missing Industry (Back-fill to cover top rows)
df['Industry'] = df['Industry'].bfill() # Fills UPWARDS
df['Industry'] = df['Industry'].ffill() # Fills DOWNWARDS (just in case)

# 3. Fix "Enquiry Box" text in Phone/Email
# Replace the junk text with NaN (empty) so we can fill it with real data
junk_text = ["ENQUIRY BOX_ Whats UP", "ENQUIRY BOX_ e mail to"]
df.replace(junk_text, np.nan, inplace=True)

# Now fill the real contact info UPWARDS to cover those rows
df[['Whatsapp', 'Email']] = df[['Whatsapp', 'Email']].bfill()

# 4. Fix Typos (Optional)
df['Item_Name'] = df['Item_Name'].str.replace('Autonomus', 'Autonomous', regex=False)

# 5. Save the Perfected File
df.to_excel(output_path, index=False)

print("✅ Mismatches Fixed!")
print(f"📂 Saved to: {output_path}")
print("\n--- Preview of Fixed Top Rows ---")
print(df.head())

✅ Mismatches Fixed!
📂 Saved to: C:\Users\dhvani\Downloads\Corrected_Final_Data.xlsx

--- Preview of Fixed Top Rows ---
                    Industry     Type  \
0  OIL & GAS Pipings systems  Project   
1  OIL & GAS Pipings systems  Project   
2  OIL & GAS Pipings systems  Project   
3  OIL & GAS Pipings systems  Product   
4  OIL & GAS Pipings systems  Product   

                                      Item_Name  \
0  NG FACTORY PIPELINES AND SKIDS INSTALLATIONS   
1    LNG STORAGE TANKS AND SYSTEM INSTALLATIONS   
2                 NITROGEN  & OXYGEN GENERATORS   
3                                         Pipes   
4                             Valves & Fittings   

                                        Primary_Link  \
0                         https://pakmon.vercel.app/   
1                         https://pakmon.vercel.app/   
2  https://www.forevergasindustry.com/en-us/products   
3                                                NaN   
4             https://www.g-bee.de/products#opt